In [1]:
'''poker engine'''

# pylint: disable=E1101, E1601, W0612

import gc
import mysql.connector
from pypokerengine.api.emulator import Emulator

import funcs_db

In [2]:
gc.collect()

20

# OPEN CONNECTION TO POKER DB 

In [3]:
poker_db = mysql.connector.connect(user='root', host='127.0.0.1', database='poker_version2')
sql_path = 'c:\\Users\\adam.sohonyai\\Documents\\GitHub\\poker_model\\sql\\version2\\'

## Delete data from database tables 

In [4]:
tables = funcs_db.sql_select_tables(poker_db=poker_db)
for table in tables:
    funcs_db.sql_delete_all(poker_db=poker_db, table=table)

# Initialization

In [5]:
emulator = Emulator()

## Create game and table

In [6]:
player_num = 6
max_round = 1
small_blind_amount = 10
ante_amount = 0

stack = 2000
players_info = {
    'uuid-0': {'name': 'player0', 'stack': stack},
    'uuid-1': {'name': 'player1', 'stack': stack},
    'uuid-2': {'name': 'player2', 'stack': stack},
    'uuid-3': {'name': 'player3', 'stack': stack},
    'uuid-4': {'name': 'player4', 'stack': stack},
    'uuid-5': {'name': 'player5', 'stack': stack}
}

summary = list()

index = funcs_db.sql_games_max_id(poker_db=poker_db)

In [7]:
emulator.set_game_rule(player_num=player_num, max_round=max_round, small_blind_amount=small_blind_amount, \
                       ante_amount=ante_amount)
initial_state = emulator.generate_initial_game_state(players_info)
for i, player in enumerate(initial_state['table'].seats.players):

    funcs_db.sql_insert_games(poker_db=poker_db, index=index + 1, player_num=player_num, small_blind_amount=small_blind_amount, \
                              ante_amount=ante_amount, uuid=player.uuid, name=player.name, stack=player.stack, position=i, \
                              card1='', card2='', hand_db_format='', flop1='', flop2='', flop3='', turn='', river='', \
                              final_stack=0)

In [8]:
game_state, events = emulator.start_new_round(initial_state)
for i, player in enumerate(game_state['table'].seats.players):
    
    card_list = [player.hole_card[0].RANK_MAP[player.hole_card[0].rank] + \
                 player.hole_card[0].SUIT_MAP[player.hole_card[0].suit], \
                 player.hole_card[1].RANK_MAP[player.hole_card[1].rank] + \
                 player.hole_card[1].SUIT_MAP[player.hole_card[1].suit]
    ]
    card_list.sort()
    card1 = card_list[0]
    card2 = card_list[1]
    if card1[-1] == card2[-1]:
        hand_db_format = card1[:-1] + card2[:-1] + 's'
    else:
        hand_db_format = card1[:-1] + card2[:-1] + 'o'

    funcs_db.sql_update_games_cards(poker_db=poker_db, index=index + 1, uuid=player.uuid, \
                                    card1=card1, card2=card2, hand_db_format=hand_db_format)

## Small blind and big blind moves

In [9]:
nr = 0
step = 0
pot = 0
for i, player in enumerate(game_state['table'].seats.players):
    
    try:
        if player.serialize()[4][0]['action'] in ['SMALLBLIND', 'BIGBLIND']:
            if player.serialize()[4][0]['action'] == 'SMALLBLIND':
                amount = small_blind_amount
            else:
                amount = 2 * small_blind_amount
            funcs_db.sql_insert_history(poker_db=poker_db, phase='preflop', nr=nr, step=step, uuid=player.serialize()[1], \
                                        position=i, stack=2000, pot=pot, flop1='', flop2='', flop3='', turn='', river='', \
                                        action=player.serialize()[4][0]['action'], amount=amount, \
                                        new_stack=2000 - amount, new_pot=pot + amount)

            nr = nr + 1
            step = step + 1
            pot = pot + amount
    except:
        pass

# Game phase

In [10]:
# poker_version2.possible_moves.bet_amount: implies the "until size" of some action, e.g. "raise til 100 
#     (already debt amount + raise amount) - emeles 100-ig (fizetendo + tovabbi novekmeny)"
# poker_version2.history.amount: net change in pot caused by some action

while events[-1]['type'] != 'event_game_finish':
    
    # data source
    data = events[-1]
    
    # calculate parameters from current game state
    phase = data['round_state']['street']
    position = data['round_state']['next_player']
    uuid = data['uuid']
    for item in data['round_state']['seats']:

        if item['uuid'] == data['uuid']:
            stack = item['stack']

    side = 0
    if data['round_state']['pot']['side'] != []:
        for amount in data['round_state']['pot']['side']:
        
            side = side + amount['amount']

    pot = data['round_state']['pot']['main']['amount'] + side
    valid_actions = funcs_db.valid_actions_check(actions=data['valid_actions'], phase=phase, position=position, \
                                                 stack=stack, small_blind_amount=small_blind_amount)
    community_cards = funcs_db.community_cards_eval(board=data['round_state']['community_card'])

    # calculate action and bet_amount and apply them on game state
    action, bet_amount, decision = funcs_db.decision_point_based_action(poker_db=poker_db, phase=phase, nr=nr, step=step, \
                                                                        position=position, stack=stack, pot=pot, \
                                                                        flop1=community_cards[0], flop2=community_cards[1], \
                                                                        flop3=community_cards[2], turn=community_cards[3], \
                                                                        river=community_cards[4], \
                                                                        valid_actions=valid_actions)
    game_state, events = emulator.apply_action(game_state=game_state, action=action, bet_amount=bet_amount)
    summary.append(decision)
    
    # calculate new parameters from current game state
    if events[-1]['type'] == 'event_game_finish':
        data = events[0]
    else:
        data = events[-1]
    
    new_phase = data['round_state']['street']
    for item in data['round_state']['seats']:

        if item['uuid'] == uuid:
            new_stack = item['stack']
    
    new_side = 0
    if data['round_state']['pot']['side'] != []:
        for amount in data['round_state']['pot']['side']:
        
            new_side = new_side + amount['amount']

    new_pot = data['round_state']['pot']['main']['amount'] + new_side
    amount = stack - new_stack
    
    # insert action in history table with calculated old and new parameters
    funcs_db.sql_insert_history(poker_db=poker_db, phase=phase, nr=nr, step=step, uuid=uuid, position=position, \
                                stack=stack, pot=pot, flop1=community_cards[0], flop2=community_cards[1], \
                                flop3=community_cards[2], turn=community_cards[3], river=community_cards[4], \
                                action=action.upper(), amount=amount, new_stack=new_stack, new_pot=new_pot)

    # increase nr parameter - nr has to be set to 0 when phase changes
    if phase == new_phase:
        nr = nr + 1
    else:
        nr = 0
    
    step = step + 1

In [11]:
funcs_db.summarize(poker_db=poker_db, final_stacks=events[-1]['players'], summary=summary)

In [12]:
# shorthands/compress feature data for:
    # pot
    # amount
    # bet amount
    # board/community cards

# prepare script for simulation - 100 rounds:
#     1. put current script in for loop -> manage seat shifts -> appropriate game info creation
#     2. re-write the script for max_round > 1

# possible simplification: create custom player based on pypokerengine modul's BasePokerPlayer class:
#     https://github.com/ishikota/PyPokerEngine